In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

## Options

In [2]:
# parse options
problem = 'twelve_pieces_process.json' # 'pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = 'results'

recompute_action_states = False
load_external_movements = False

In [3]:
# generic planning options
id_only = None # 'A2_M1' # None | 'A273_M0'

low_res = False
plan_impacted = False

#     'nonlinear',
#     'linear',
#     'id_only', # 'Compute only for movement with a specific tag, e.g. `A54_M0`.'
#     'free_motion_only', # 'Only compute free motions.'
#     'propagate_only', # 'Only do state propagation and impacted movement planning.'
solve_mode = 'linear'

viz_upon_found = False

In [4]:
# client options
viewer = True
verbose = True

debug = False
diagnosis = False

watch = True
step_sim = False

disable_env = False
reinit_tool = False

write = False
save_now = False

In [5]:
from collections import namedtuple
PlanningArguments = namedtuple('PlanningArguments', ['problem', 'debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
                                             'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

args = PlanningArguments(problem, debug, diagnosis, id_only, solve_mode, viz_upon_found, save_now, write, plan_impacted, watch, step_sim, verbose)

## Parse process from json

In [6]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, get_process_path, save_process

In [7]:
process = parse_process(problem, subdir=problem_subdir)
result_path = get_process_path(problem, subdir='results')
if len(process.movements) == 0:
    cprint('No movements found in process, trigger recompute actions.', 'red')
    recompute_action_states = True
if recompute_action_states:
    cprint('Recomputing Actions and States', 'cyan')
    recompute_action_states(process)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results\twelve_pieces_process.json


In [8]:
from copy import deepcopy
unsolved_process = deepcopy(process)

In [9]:
# # force load external if only planning for the free motions
# load_external_movements = load_external_movements or free_motion_only or id_only is not None
# if load_external_movements:
#     ext_movement_path = os.path.dirname(result_path)
#     cprint('Loading external movements from {}'.format(ext_movement_path), 'cyan')
#     process.load_external_movements(ext_movement_path)
#     if recompute_action_states:
#         save_process(process, result_path)
#         cprint('Recomputed process saved to %s' % result_path, 'green')

## Start client

In [25]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
client, robot, _ = load_RFL_world(viewer=viewer or diagnosis or view_states or watch or step_sim, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [21]:
from integral_timber_joints.planning.robot_setup import GANTRY_ARM_GROUP, GANTRY_GROUP, BARE_ARM_GROUP
from compas.robots import Joint
import numpy as np

joint_names = robot.get_configurable_joint_names(group=GANTRY_ARM_GROUP)
joint_types = robot.get_joint_types_by_names(joint_names)
# 0.1 rad = 5.7 deg
joint_jump_threshold = {jt_name : np.pi/6 \
        if jt_type in [Joint.REVOLUTE, Joint.CONTINUOUS] else 0.1 \
        for jt_name, jt_type in zip(joint_names, joint_types)}

options = {
    'distance_threshold' : 0.0012,
    'frame_jump_tolerance' : 0.0012,
    'verbose' : verbose,
    'jump_threshold' : joint_jump_threshold,
#     'max_distance' : args.max_distance,
}

# Plan movements for a beam

In [26]:
from copy import copy, deepcopy
import time

# start_time = time.time()
# process = copy(unsolved_process)
# print('Copy time: {:.5f}'.format(time.time() - start_time))

start_time = time.time()
process = deepcopy(unsolved_process)
print('Deepcopy time: {:.5f}'.format(time.time() - start_time))

Deepcopy time: 4.29753


In [22]:
# beam_id = 'b10'
# process.get_movement_summary_by_beam_id(beam_id)

In [27]:
from integral_timber_joints.planning.run import compute_movements_for_beam_id
# beam_id = process.get_beam_id_from_movement_id(args.id_only)

beam_id = 'b8'
# options['movement_id_range'] = range(36,41) # 35

options.update({
    'debug' : False,
    'diagnosis' : False,
    'low_res' : False,
})

# PlanningArguments = namedtuple('PlanningArguments', ['problem', 'debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
#                                              'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

args = PlanningArguments(problem, False, False, id_only, 'linear', viz_upon_found, save_now, write, 
                         plan_impacted, watch, False, verbose)

compute_movements_for_beam_id(client, robot, process, beam_id, args, options=options)




* compute movement ids: ['A64_M0', 'A64_M1', 'A64_M2', 'A64_M3', 'A64_M4', 'A64_M5', 'A65_M0', 'A65_M1', 'A65_M2', 'A65_M3', 'A65_M4', 'A65_M5', 'A66_M0', 'A66_M1', 'A66_M2', 'A66_M3', 'A66_M4', 'A66_M5', 'A67_M0', 'A67_M1', 'A67_M2', 'A67_M3', 'A67_M4', 'A67_M5', 'A68_M0', 'A68_M1', 'A68_M2', 'A68_M3', 'A68_M4', 'A69_M0', 'A69_M1', 'A69_M2', 'A69_M3', 'A69_M4', 'A69_M5', 'A70_M0', 'A70_M1', 'A70_M2', 'A70_M3', 'A70_M4', 'A70_M5', 'A71_M0', 'A71_M1', 'A71_M2', 'A71_M3', 'A71_M4', 'A72_M0', 'A72_M1', 'A72_M2', 'A72_M3', 'A72_M4', 'A72_M5', 'A72_M6', 'A73_M0', 'A73_M1', 'A73_M2', 'A73_M3', 'A73_M4', 'A73_M5', 'A74_M0', 'A74_M1', 'A74_M2', 'A74_M3', 'A74_M4', 'A74_M5', 'A74_M6', 'A75_M0', 'A75_M1', 'A75_M2', 'A75_M3', 'A75_M4', 'A75_M5']
----------
(0)
RoboticFreeMovement(#A64_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to get clamp., traj 0)
FreeMovement: Robot start conf is NOT specified in RoboticFreeMovement(#A64_M0, Free Move reach Storage Approach Frame of CL3 ('c2'

RRT connect: 55 iterations, 355 nodes
Free movement found for RoboticFreeMovement(#A66_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to get clamp., traj 0)!
~~~~~
	Propagate states for (12) : RoboticFreeMovement(#A66_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to get clamp., traj 1)
	$ Impacted (forward): (13) RoboticLinearMovement(#A66_M1, Linear Advance to Storage Frame of CL3 ('c1'), to get tool., traj 0)
----------
(13)
RoboticLinearMovement(#A66_M1, Linear Advance to Storage Frame of CL3 ('c1'), to get tool., traj 0)
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Conf disagreement found, but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.
Joint #bridge1_joint_EA_X (revolution) jump: 0.4891 | tol: 0.1000
Joint #robot11_joint_EA_Y (revolution) jump: 0.5828 | tol: 0.

RRT connect: 28 iterations, 202 nodes
Free movement found for RoboticFreeMovement(#A68_M0, Free Move reach Storage Approach Frame of PG500 ('g1'), to get tool., traj 0)!
~~~~~
	Propagate states for (24) : RoboticFreeMovement(#A68_M0, Free Move reach Storage Approach Frame of PG500 ('g1'), to get tool., traj 1)
	$ Impacted (forward): (25) RoboticLinearMovement(#A68_M1, Linear Advance to Storage Frame of PG500 ('g1'), to get tool., traj 0)
----------
(25)
RoboticLinearMovement(#A68_M1, Linear Advance to Storage Frame of PG500 ('g1'), to get tool., traj 0)
end conf FK inconsistent (0.00002 m) with given current frame in end state.
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Conf disagreement found, but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.
Joint #bridge1_joint_EA_X (revolution) jump: 

Conf disagreement found, but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 1 path failure (by IterativeIK) over 2 samples.
~~~~~
	Propagate states for (38) : RoboticClampSyncLinearMovement(#A70_M3, Robot and Clamps (['c2', 'c1']) syncronously move to clamp Beam ('b8'), traj 1)
	- Altered (backward): (37) ClampsJawMovement(#A70_M2, Clamps (['c2', 'c1']) close slightly to touch Beam ('b8'))
	- Altered (forward): (39) RoboticDigitalOutput(#A70_M4, Open Gripper ('g1') and let go of Beam ('b8'))
	$ Impacted (forward): (40) RoboticLinearMovement(#A70_M5, Linear retract after placing Beam ('b8'), traj 0)
----------
(39)
----------
(40)
RoboticLinearMovement(#A70_M5, Linear retract after placing Beam ('b8'), traj 0)
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Conf disagreement found, but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.
~~~~~
	Pr

One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #2
Conf disagreement found, but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 2 path failure (by IterativeIK) over 3 samples.
~~~~~
	Propagate states for (52) : RoboticLinearMovement(#A72_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c2') from structure., traj 1)
	$ Impacted (forward): (53) RoboticFreeMovement(#A73_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 0)
----------
(53)
RoboticFreeMovement(#A73_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 0)
FreeMovement: Robot end conf is NOT specified in RoboticFreeMovement(#A73_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 0), we 

Conf disagreement found, but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 1 path failure (by IterativeIK) over 2 samples.
~~~~~
	Propagate states for (65) : RoboticLinearMovement(#A74_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c1') from structure., traj 1)
	$ Impacted (forward): (66) RoboticFreeMovement(#A75_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to place clamp in storage., traj 0)
----------
(66)
RoboticFreeMovement(#A75_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to place clamp in storage., traj 0)
FreeMovement: Robot end conf is NOT specified in RoboticFreeMovement(#A75_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to place clamp in storage., traj 0), we will sample an IK conf based on the given t0cp frame.
Warning! Go back to the command line now!
RRT connect: 48 iterations, 402 nodes
Free movement found for RoboticFreeMovement(#A75_M0, Free Move reach Storage Approach Frame of CL3 (

===
Viz:
===
Viz:
RoboticClampSyncLinearMovement(#A70_M3, Robot and Clamps (['c2', 'c1']) syncronously move to clamp Beam ('b8'), traj 1)
===
Viz:
===
Viz:
RoboticLinearMovement(#A70_M5, Linear retract after placing Beam ('b8'), traj 1)
===
Viz:
RoboticFreeMovement(#A71_M0, Free Move to reach Storage Approach Frame of PG500 ('g1'), to place tool in storage., traj 1)
===
Viz:
RoboticLinearMovement(#A71_M1, Linear Advance to Storage Frame of PG500 ('g1'), to place tool in storage., traj 1)
===
Viz:
===
Viz:
===
Viz:
RoboticLinearMovement(#A71_M4, Linear Retract from storage after placing PG500 ('g1') in storage, traj 1)
===
Viz:
RoboticFreeMovement(#A72_M0, Free Move to reach CL3 ('c2') to detach it from structure., traj 1)
===
Viz:
RoboticLinearMovement(#A72_M1, Linear Advance to mate toolchanger of CL3 ('c2') to detach it from structure., traj 1)
===
Viz:
===
Viz:
===
Viz:
===
Viz:
RoboticLinearMovement(#A72_M5, Linear Retract 1 of 2 from storage after picking up CL3 ('c2') from struct

True

In [18]:
from integral_timber_joints.planning.parsing import save_process_and_movements

beam_all_movements = process.get_movements_by_beam_id(beam_id)
altered_movements = [beam_all_movements[mid] for mid in options['movement_id_range']]
save_process_and_movements(problem, process, altered_movements, save_temp=True)

KeyError: 'movement_id_range'

# Disconnect client

In [28]:
client.disconnect()

# Plan only one movement

In [29]:
# if id_only:
#     beam_id = process.get_beam_id_from_movement_id(id_only)
#     process.get_movement_summary_by_beam_id(beam_id)

In [39]:
from integral_timber_joints.planning.stream import compute_free_movement, compute_linear_movement
from integral_timber_joints.planning.solve import compute_movement

chosen_m = process.get_movement_by_movement_id(id_only)
compute_movement(client, robot, process, chosen_m, options=lm_options, diagnosis=diagnosis)

RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)
end conf FK inconsistent (0.00005 m) with given current frame in end state.
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.


True

In [40]:
from integral_timber_joints.planning.visualization import visualize_movement_trajectory

with pp.WorldSaver():
    visualize_movement_trajectory(client, robot, process, chosen_m, step_sim=True)

===
Viz:
RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 
